# Import Needed Modules

In [2]:
import torch
import numpy as np
import torch.nn as nn

# X & Y for TensorDataset and DataLoader

In [7]:
#inputs columns: [temperature , Rainfall , Humidity]
inputs= np.array([[73,67,43],    [91,88,64],    [87,137,58],
                  [102,43,37],   [69,96,70],    [73,67,43],
                  [91,88,64],    [87,137,58],   [102,43,37],
                  [69,96,70],    [73,67,43],    [91,88,64],
                  [87,137,58],   [102,43,37],   [69,96,70]],dtype='float32')

#Targets columns: apples , Oranges
targets = np.array([[56,70],   [81,101],  [119,133],
                    [22,37],   [103,119], [56,70],  
                    [81,101],  [119,133], [22,37], 
                    [103,119], [56,70],   [81,101], 
                    [119,133], [22,37],   [103,119]],
                   dtype='float32')

#turn arrays into tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [9]:
#create tensordata set to work with inputs and targets together
from torch.utils.data import TensorDataset

#define dataset
train_ds = TensorDataset(inputs , targets)

In [14]:
#can be accessed by indexing
train_ds[0]

(tensor([73., 67., 43.]), tensor([56., 70.]))

In [15]:
#create a Dataloader to shuffle and split data into batches of a predifined size
from torch.utils.data import DataLoader

#predefined batch size
chosen_batch = 5
#define Dataloader
train_dl = DataLoader(train_ds ,chosen_batch , shuffle=True )

In [28]:
for Xb ,yb in train_dl:
    print(Xb , yb)

tensor([[ 87., 137.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 87., 137.,  58.],
        [ 91.,  88.,  64.]]) tensor([[119., 133.],
        [103., 119.],
        [ 56.,  70.],
        [119., 133.],
        [ 81., 101.]])
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]]) tensor([[ 22.,  37.],
        [ 81., 101.],
        [ 56.,  70.],
        [ 81., 101.],
        [ 22.,  37.]])
tensor([[ 87., 137.,  58.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.]]) tensor([[119., 133.],
        [103., 119.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.]])


# nn.Linear : for initializing weights/beta's and bias/constants

In [30]:
#define model
#nn.linear takes(len of input cols , len of target cols) to randomly initialize weights & biases
lin_model = nn.Linear(3 , 2)
print(lin_model.weight)
print(lin_model.bias)

Parameter containing:
tensor([[ 0.1143, -0.3246,  0.4964],
        [ 0.1412,  0.4352,  0.4377]], requires_grad=True)
Parameter containing:
tensor([-0.3422,  0.5470], requires_grad=True)


In [31]:
#to access Parameters of all torch models
list(lin_model.parameters())

[Parameter containing:
 tensor([[ 0.1143, -0.3246,  0.4964],
         [ 0.1412,  0.4352,  0.4377]], requires_grad=True),
 Parameter containing:
 tensor([-0.3422,  0.5470], requires_grad=True)]

In [32]:
#Generate Predictions of untrained model
first_pred= lin_model(inputs)

# InBuilt Loss Function

In [33]:
import torch.nn.functional as F

In [34]:
#define loss function
loss_fn = F.mse_loss

In [35]:
#compute the loss
loss=loss_fn(first_pred , targets)
loss

tensor(3435.6235, grad_fn=<MseLossBackward>)

# Optimizer

lets use optim.SGD: Stochastic Gradient Descent

In [36]:
#Define optimizer
Optimizer = torch.optim.SGD(lin_model.parameters() , lr=0.00005)

# Model Trainer

Using gradient descent optimazation algorithm, Build a training function with these steps
- Generate predictions
- Calculate the loss
- Set backward on loss and Compute the gradients w.r.t to the weights and bias
- Adjust the weights by subtracting a small quantity proportional to the gradient
- Reset the gradients back to zero

In [59]:
#creat a utility function ton train model
def model_trainer(model , train_dl , num_of_epoch,loss_func , opt):
    
    #num of training periods
    for epoch in range(num_of_epoch):
        
        #unpack X and y batches
        for Xb, yb in train_dl:
            
            #Generate predictions
            pred = model(Xb)
            
            #Calculate loss
            loss=loss_func(pred , yb)
            
            #Compute gradients
            loss.backward()
            
            #update model parameters
            opt.step()
            
            #Reset model parameters to zero
            opt.zero_grad()
            
        #Print Progress
        if (epoch+1) % 10 == 0:
            print('{}/{} , Loss:{}'.format(epoch+1, num_of_epoch , loss))

In [60]:
model_trainer(lin_model , train_dl, 100 , loss_fn , Optimizer)

10/100 , Loss:1.0553748607635498
20/100 , Loss:0.7759954333305359
30/100 , Loss:0.41868266463279724
40/100 , Loss:0.6888500452041626
50/100 , Loss:0.9380098581314087
60/100 , Loss:0.3434423804283142
70/100 , Loss:0.36811771988868713
80/100 , Loss:0.7129873037338257
90/100 , Loss:0.5350072383880615
100/100 , Loss:0.6088463068008423


In [61]:
lastpred=lin_model(inputs)
lastpred

tensor([[ 56.9303,  70.0099],
        [ 82.5030, 100.5592],
        [119.1712, 132.9682],
        [ 21.4066,  37.1271],
        [102.2971, 119.2036],
        [ 56.9303,  70.0099],
        [ 82.5030, 100.5592],
        [119.1712, 132.9682],
        [ 21.4066,  37.1271],
        [102.2971, 119.2036],
        [ 56.9303,  70.0099],
        [ 82.5030, 100.5592],
        [119.1712, 132.9682],
        [ 21.4066,  37.1271],
        [102.2971, 119.2036]], grad_fn=<AddmmBackward>)

In [53]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [1]:
import jovian

jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Uploading notebook..
[jovian] Capturing environment..


TypeError: invalid cmd type (<class 'NoneType'>, expected string)